Teknik Algoritma Genetik

In [ ]:
#Import Library
import numpy as np

#Import Operator
import random
from array import *

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Open input file
infile = open('berlin52.tsp', 'r')

# Read instance header
Name = infile.readline().strip().split()[1] # NAME
FileType = infile.readline().strip().split()[1] # TYPE
Comment = infile.readline().strip().split()[1] # COMMENT
Dimension = infile.readline().strip().split()[1] # DIMENSION
EdgeWeightType = infile.readline().strip().split()[1] # EDGE_WEIGHT_TYPE
infile.readline()

# Read node list
daftarLokasi = []
N = int(Dimension)
for i in range(0, N):
    x,y = infile.readline().strip().split()[1:]
    daftarLokasi.append([float(x), float(y)])

# Close input file
infile.close()
print("  [X]", "     [Y]")

for i in daftarLokasi:
    for j in i:
        print("[",j,end = "]  ")
    print()

jumlahLokasi =  len(daftarLokasi)   
print("")
print("total lokasi",jumlahLokasi)

  [X]      [Y]
[ 565.0]  [ 575.0]  
[ 25.0]  [ 185.0]  
[ 345.0]  [ 750.0]  
[ 945.0]  [ 685.0]  
[ 845.0]  [ 655.0]  
[ 880.0]  [ 660.0]  
[ 25.0]  [ 230.0]  
[ 525.0]  [ 1000.0]  
[ 580.0]  [ 1175.0]  
[ 650.0]  [ 1130.0]  
[ 1605.0]  [ 620.0]  
[ 1220.0]  [ 580.0]  
[ 1465.0]  [ 200.0]  
[ 1530.0]  [ 5.0]  
[ 845.0]  [ 680.0]  
[ 725.0]  [ 370.0]  
[ 145.0]  [ 665.0]  
[ 415.0]  [ 635.0]  
[ 510.0]  [ 875.0]  
[ 560.0]  [ 365.0]  
[ 300.0]  [ 465.0]  
[ 520.0]  [ 585.0]  
[ 480.0]  [ 415.0]  
[ 835.0]  [ 625.0]  
[ 975.0]  [ 580.0]  
[ 1215.0]  [ 245.0]  
[ 1320.0]  [ 315.0]  
[ 1250.0]  [ 400.0]  
[ 660.0]  [ 180.0]  
[ 410.0]  [ 250.0]  
[ 420.0]  [ 555.0]  
[ 575.0]  [ 665.0]  
[ 1150.0]  [ 1160.0]  
[ 700.0]  [ 580.0]  
[ 685.0]  [ 595.0]  
[ 685.0]  [ 610.0]  
[ 770.0]  [ 610.0]  
[ 795.0]  [ 645.0]  
[ 720.0]  [ 635.0]  
[ 760.0]  [ 650.0]  
[ 475.0]  [ 960.0]  
[ 95.0]  [ 260.0]  
[ 875.0]  [ 920.0]  
[ 700.0]  [ 500.0]  
[ 555.0]  [ 815.0]  
[ 830.0]  [ 485.0]  
[ 1170.0]  [

In [ ]:
#Inisialisasi Populasi secara acak

#Array P diinisialisasi sebagai array integer

#Solusi awal dibuat dengan random permutation

def initPopulasi(ukuranPopulasi,jumlahLokasi):
  P = np.empty((ukuranPopulasi,jumlahLokasi))
  P = P.astype('int')
  for i in range(ukuranPopulasi):
    I = np.random.permutation(jumlahLokasi)
    for j in range(jumlahLokasi):
      P[i][j]=I[j]
  return P

ukuranPopulasi = 10
P = initPopulasi (ukuranPopulasi,jumlahLokasi)
print(P)

[[39  9 13 42 45 26 11 49 22  3 32 35 36  7 27 41  6 47 28 14  8 12 37 48
   2 17 15 24 21 19 40  4  1 38 30 46 43 34 29  0 20  5 33 51 23 10 16 44
  25 18 50 31]
 [ 5 15 40 19 29  9 37 30 27 20 12 32  6 22 28 34 49 31 16  3 36  7 21 33
   2  0 50 35 13 48 46 23 42  8 14 24 11 26 17 47  4 45 38 25 51  1 43 39
  18 41 44 10]
 [ 2 23 19 26 44 14 34 27 37 21 17 22 35 47  7  9 24 18 39 42  5 16 51 12
  45 43 31  6 20 32 36  3 40 49 10 38 33 11 50  1  8 46 30 29 15  4 13 25
  28  0 48 41]
 [20 27 48  0  8 36 19 35 30 47 16 42  3 50  4 29  2 23 44  6 45 37 13 15
  21 26  1  5 25 17 38 39 28 49 10 24 31  7 51  9 11 32 12 34 18 14 46 43
  33 22 41 40]
 [46 24 12 25 37 36  2 50  9 19 39 47 15 29  8 14 17 43 18 30 45 42 22 28
  23 44 20 21  0 48 32 41 33  3 40 51  1 27  7 34 10 26 31 11 13 16  5  4
   6 35 49 38]
 [35  8 14 39  5 26 41 18 22 16 42 40 50 28 21 43 24 48 13  1 10  7 20 11
  15  4 46 30 25  2 49 12 31 38 34 44 29 19  0 33 27 36 45 23  6 17 51 32
   9  3 47 37]
 [22 32  8  3 20 42 24

In [ ]:
#Membuat Fungsi Objektif

#Jarak yang diukur adalah Jarak Euclidian

def fungsiObjectif(populasi,daftarLokasi):
  sz = populasi.shape
  ukuranPopulasi = sz[0]
  jumlahLokasi = sz[1]

  matrikJarak = np.empty((ukuranPopulasi))
  matrikJarak = matrikJarak.astype('float')
  n = len(daftarLokasi)

  for i in range(ukuranPopulasi):
    jarak = 0
    for j in range(n-1):
      jrkX = daftarLokasi[populasi[i][j]][0]-daftarLokasi[populasi[i][j+1]][0]
      jrkY = daftarLokasi[populasi[i][j]][1]-daftarLokasi[populasi[i][j+1]][1]
      d = np.sqrt(jrkX**2 + jrkY**2 )   
      jarak = jarak  + d 
      
    dn =  np.sqrt((daftarLokasi[populasi[i][n-1]][0]-daftarLokasi[populasi[i][0]][0])**2+ (daftarLokasi[populasi[i][n-1]][1]-daftarLokasi[populasi[i][0]][1])**2 )
    jarak = jarak  + dn
    matrikJarak[i] = jarak

  return matrikJarak

fitP = fungsiObjectif(P,daftarLokasi)
print(fitP)
#print(fitP.dtype)

[31221.26393461 30083.48559839 27760.7302374  30905.62712301
 30675.7794309  30756.89117644 28741.11980409 29555.83434119
 27298.00404395 29850.99258996]


In [ ]:
#Seleksi menggunakan Metode Tournamen

#Ukuran Tournamen k = 2

#Jumlah Induk yang dipilih adalah 2

def seleksi (P,fsObj):
  jmlInduk = 2
  sz = P.shape
  ukuranPopulasi = sz[0]
  jumlahLokasi = sz[1]

  Induk = np.empty((jmlInduk,jumlahLokasi))
  Induk = Induk.astype('int')
  
  for i in range (2):
    #memilih dua individu secara acak 
    r1 = random.randint(0,ukuranPopulasi-1)
    r2 = random.randint(0,ukuranPopulasi-1)
    
    if  fsObj[r1] < fsObj [r2]:
      for j in range(jumlahLokasi):
        Induk [i][j] = P[r1][j]
    else:
      for j in range(jumlahLokasi):
        Induk [i][j] = P[r2][j]
  return Induk

Induk = seleksi(P,fitP)
print(Induk)

[[39  9 13 42 45 26 11 49 22  3 32 35 36  7 27 41  6 47 28 14  8 12 37 48
   2 17 15 24 21 19 40  4  1 38 30 46 43 34 29  0 20  5 33 51 23 10 16 44
  25 18 50 31]
 [39 29 10 31 19  1 20 40 23 41  7 21  8 15 48 46 22  9 26 25 30 50  0 43
   2 28  6 37 33 38 27 24 35 42 44 51 14 32  5 18 34  3 16 49 47 17 13 12
  45 11 36  4]]


In [ ]:
#Rekombinasi menggunakan Metode Order Crossover (OX) Jumlah anak yang dihasilkan adalah 2
#crossover 

def PMX(Induk):
  Induk = seleksi(P,fitP)
  sz = Induk.shape
  brs = sz[0]
  kol = sz[1]

  I1 = np.empty((kol))
  I2 = np.empty((kol))
  A1 = np.empty((kol))
  A2 = np.empty((kol))
  Anak = np.empty((2,kol))
  Anak = Anak.astype('int')

  for i in range(kol):
    I1[i]=Induk[0][i]
    I2[i]=Induk[1][i]

  r1 = random.randint(1,kol-2)

  for i0 in range(r1): 
    A1[i0] = I1[i0]
    A2[i0] = I2[i0]

  batasCek1 = r1
  batasCek2 = r1

  for i in range(kol):
    m = (i+r1)%kol

    n1 = I1[m] 
    n2 = I2[m]

    #mengecek apakah Lokasi tertentu sudah ada di anak

    idxAda1 = 0
    idxAda2 = 0
    for j1 in range(batasCek1): 
      
      if batasCek1>(kol):
        idxAda1 = 1

      if A1[j1] == n2: 
        idxAda1 = 1
            
    for j2 in range(batasCek2): 
      
      if batasCek2>(kol):
        idxAda2 = 1

      if A2[j2] == n1: 
        idxAda2 = 1

    if idxAda1 == 0:
      A1[batasCek1] = n2
      batasCek1 = batasCek1 +1
   
    if idxAda2 == 0:
      A2[batasCek2] = n1
      batasCek2 = batasCek2 +1
   
  for i in range(kol):
    Anak[0][i]=A1[i]
    Anak[1][i]=A2[i]

  return Anak

AnakCX = PMX(Induk)
print("Anak :",AnakCX)

Anak : [[39 29 10 31 19  1 20 35 30 47 16 42  3 50  4  2 23 44  6 45 37 13 15 21
  26  5 25 17 38 28 49 24  7 51  9 11 32 12 34 18 14 46 43 33 22 41 40 27
  48  0  8 36]
 [20 27 48  0  8 36 19 40 23 41  7 21 15 46 22  9 26 25 30 50 43  2 28  6
  37 33 38 24 35 42 44 51 14 32  5 18 34  3 16 49 47 17 13 12 45 11  4 39
  29 10 31  1]]


Mutasi dengan Swap

In [ ]:
#Index Lokasi yang di Swap pada kedua anak sama

def MutasiSwap(Induk):
  sz = Induk.shape
  brs = sz[0]
  kol = sz[1]

  Anak = np.empty((2,kol))
  Anak = Anak.astype('int')

  Anak = Induk

  r1 = random.randint(0,kol-1)
  r2 = random.randint(0,kol-1)

  while r1 == r2:
    r2 = random.randint(0,kol-1)
  
  if r1 < r2:
    rs1 = r1
    rs2 = r2
  else:
    rs1 = r2
    rs2 = r1

  I1rs1 = Induk [0][rs1]
  I1rs2 = Induk [0][rs2]
  I2rs1 = Induk [1][rs1]
  I2rs2 = Induk [1][rs2]

  Anak[0][rs1] = I1rs2
  Anak[0][rs2] = I1rs1
  Anak[1][rs1] = I2rs2
  Anak[1][rs2] = I2rs1

  return Anak

print("AnakCX :",AnakCX)
AnakM = MutasiSwap(AnakCX)
print("AnakM :",AnakM)

AnakCX : [[39 29 10 31 19  1 20 35 30 47 16 42  3 50  4  2 23 44  6 45 37 13 15 21
  26  5 25 17 38 28 49 24  7 51  9 11 32 12 34 18 14 46 43 33 22 41 40 27
  48  0  8 36]
 [20 27 48  0  8 36 19 40 23 41  7 21 15 46 22  9 26 25 30 50 43  2 28  6
  37 33 38 24 35 42 44 51 14 32  5 18 34  3 16 49 47 17 13 12 45 11  4 39
  29 10 31  1]]
AnakM : [[39 29 10 31 19  1 20 35 30 47 16 42  3 50  4 13 23 44  6 45 37  2 15 21
  26  5 25 17 38 28 49 24  7 51  9 11 32 12 34 18 14 46 43 33 22 41 40 27
  48  0  8 36]
 [20 27 48  0  8 36 19 40 23 41  7 21 15 46 22  2 26 25 30 50 43  9 28  6
  37 33 38 24 35 42 44 51 14 32  5 18 34  3 16 49 47 17 13 12 45 11  4 39
  29 10 31  1]]


Elitism (steady state replacement)

In [ ]:
def Elitism(P,Anak,fitP,fitAnak):
  szP = P.shape
  brsP = szP[0]
  kolP = szP[1]
  szA = Anak.shape
  brsA = szA[0]
  kolA = szA[1]

  #print (fitP)

  for i in range(brsA):
    iJelek = fitP.max()
    idxJelek = fitP.argmax()

    if fitAnak[i]<iJelek:
      fitP[idxJelek] = fitAnak[i]
      for j in range (kolP):
        P[idxJelek][j] = Anak[i][j]
  
  return [P,fitP]

fitAnak = fungsiObjectif(AnakM,daftarLokasi)
print(P)
print(AnakM)
print(fitP)
print(fitAnak)
P = Elitism (P,AnakM,fitP,fitAnak)
print(P)
print(fitP)

[[39  9 13 42 45 26 11 49 22  3 32 35 36  7 27 41  6 47 28 14  8 12 37 48
   2 17 15 24 21 19 40  4  1 38 30 46 43 34 29  0 20  5 33 51 23 10 16 44
  25 18 50 31]
 [ 5 15 40 19 29  9 37 30 27 20 12 32  6 22 28 34 49 31 16  3 36  7 21 33
   2  0 50 35 13 48 46 23 42  8 14 24 11 26 17 47  4 45 38 25 51  1 43 39
  18 41 44 10]
 [ 2 23 19 26 44 14 34 27 37 21 17 22 35 47  7  9 24 18 39 42  5 16 51 12
  45 43 31  6 20 32 36  3 40 49 10 38 33 11 50  1  8 46 30 29 15  4 13 25
  28  0 48 41]
 [20 27 48  0  8 36 19 35 30 47 16 42  3 50  4 29  2 23 44  6 45 37 13 15
  21 26  1  5 25 17 38 39 28 49 10 24 31  7 51  9 11 32 12 34 18 14 46 43
  33 22 41 40]
 [46 24 12 25 37 36  2 50  9 19 39 47 15 29  8 14 17 43 18 30 45 42 22 28
  23 44 20 21  0 48 32 41 33  3 40 51  1 27  7 34 10 26 31 11 13 16  5  4
   6 35 49 38]
 [35  8 14 39  5 26 41 18 22 16 42 40 50 28 21 43 24 48 13  1 10  7 20 11
  15  4 46 30 25  2 49 12 31 38 34 44 29 19  0 33 27 36 45 23  6 17 51 32
   9  3 47 37]
 [22 32  8  3 20 42 24

Prosedur GA

In [ ]:
#Inisialisai Parameter
PCX = 0.95
Pm = 0.01
ukuranPopulasi = 10
max_generasi = ((2000-ukuranPopulasi)//2)

#Inisialisasi Populasi

P = initPopulasi (ukuranPopulasi,jumlahLokasi)
print("P0 :",P)
print("=========================================================================")
#melakukan Evaluasi P

fitP = fungsiObjectif(P,daftarLokasi)
print("fitness populasi awal :",fitP)
print("fitness terbaik awal :",fitP.min())
print("rute awal :",P[fitP.argmin()])
print("=========================================================================")

for i in range(max_generasi):
  Induk = seleksi(P,fitP)
  #print("Induk :",Induk)

  Anak = Induk

  rxo = random.random()

  if rxo < PCX:
    Anak = PMX(Induk)

    rm = random.random()
    if rm < Pm:
      Anak = MutasiSwap(Anak)

    fitAnak = fungsiObjectif(Anak,daftarLokasi)
    #print("fitAnak :",fitAnak)

    [P,fitP] = Elitism (P,Anak,fitP,fitAnak)

print("fitness populasi akhir :",fitP)
print("fitness akhir :",fitP.min())
print("rute akhir :",P[fitP.argmin()])


P0 : [[51 18 37 25 27 11 43 46 30 21 31 48 40 42  0 33 16 19  1 23  7 38  9 50
  26  4  2 36 13 29 17 24 35 15 41 14 47  6  3 20 22 34 28 44 39  8 49 45
  32 12 10  5]
 [38 42 50 49  2 31 47 15 46  7 12 40 23 37 17 45 22  8  4 28 18  0 10 26
   5 34  3 24 27 11 41 51  9  1 21 29  6 13 32 16 20 44 39 30 14 35 19 43
  48 36 33 25]
 [43 25 15 32 36  2 17 10 24 33 48  1 16 27 28  3 49  7 35 46 50 26 20  0
  31 47 41 12 30 29 39 19 11  4 45  5 14 42 18  8  6 38 23 21 22 40 51  9
  34 37 44 13]
 [26  3 33 38 22 16 39  4 29 30 25 19 24 34 51  9 12 36 43 35 41  0 13 20
  17 40 14 46 45 47 10 23  8  1 21  5 37  6 32 50 44 31 42 18  2 11  7 48
  49 15 28 27]
 [40 43  5  0 46 29  6 11 30 12 24 49 50 39 47 21 27 33 38 41 35 32 25  8
  31 45 51 20 14  9  3 23 36  7 17 28 37 26 42 22 13 15  4 44 10  2 34  1
  18 19 48 16]
 [29 11 30 45 19 34 49 40 36  8 17 31 50 42 43 38 23 20 27 33  7 10 15 12
   9 24 48 47 21  5 35  1 44 51 37 26 46 32 25 18 22  0  2 13 14 39  6 16
  28  4  3 41]
 [ 5 12 44 37 30 